# importing the dependencies

In [1]:
import numpy as np 
import pandas as pd 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/absurd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stop words in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# data collection and preprocessing

In [4]:
# loading the dataset to pandas dataframe
news_data = pd.read_csv('train.csv.zip')

In [5]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
news_data.shape

(20800, 5)

In [7]:
# counting the number of missing value
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# replacing the null value with empty string 
news_data = news_data.fillna('')

In [10]:
news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
# merging the author name and news title
news_data['content']= news_data['author']+' '+ news_data['title']

In [12]:
print(news_data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
# separating the data
x = news_data.drop(columns='label', axis=1)
y = news_data['label']

In [14]:
print(x)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [15]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


# stemming

In [26]:
port_stem = PorterStemmer()

In [27]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-A]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content =' '.join(stemmed_content)
    return stemmed_content

In [28]:
news_data['content'] = news_data['content'].apply(stemming)

In [29]:
print(news_data['content'])

0        arrel ucu ous em aid e idn ven ee omey etter n...
1           aniel lynn illari linton ig oman ampu reitbart
2                  onsortiumnew com hy ruth ight et ou ire
3        essica urkiss ivilian ill n ingl airstrik ave ...
4        oward ortnoy ranian woman jail fiction unpubli...
                               ...                        
20795       erom udson apper rump oster hild hite upremaci
20796    enjamin offman layoff chedul atchup dd ew ork ime
20797    ichael de la erc achel abram aci aid eceiv ake...
20798         alex ansari ussia old arallel xercis n alkan
20799                             avid wanson hat eep aliv
Name: content, Length: 20800, dtype: object


In [30]:
# separating the data and label
x = news_data['content'].values
y = news_data['label'].values

In [31]:
print(x)

['arrel ucu ous em aid e idn ven ee omey etter ntil ason haffetz weet'
 'aniel lynn illari linton ig oman ampu reitbart'
 'onsortiumnew com hy ruth ight et ou ire' ...
 'ichael de la erc achel abram aci aid eceiv akeov approach udson ay ew ork ime'
 'alex ansari ussia old arallel xercis n alkan' 'avid wanson hat eep aliv']


In [32]:
print(y)

[1 0 1 ... 0 1 1]


In [33]:
y.shape

(20800,)

In [34]:
# transforming the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [35]:
print(x)

  (0, 16133)	0.2751136106666613
  (0, 15914)	0.25721451035053505
  (0, 15036)	0.330956879736425
  (0, 11858)	0.2018697211989392
  (0, 10896)	0.22863312602540892
  (0, 10261)	0.3022365599314355
  (0, 7414)	0.27837634506463166
  (0, 6533)	0.34682278796528043
  (0, 5878)	0.25344000620589097
  (0, 4989)	0.24918253458902975
  (0, 4532)	0.21151494630800288
  (0, 2281)	0.21895854596793288
  (0, 2074)	0.31732758849472026
  (0, 588)	0.21625884494417633
  (1, 13026)	0.19892561984000545
  (1, 10841)	0.3901807845495569
  (1, 9232)	0.4577334143572521
  (1, 8988)	0.24717748739783138
  (1, 7668)	0.2459062303383013
  (1, 7495)	0.35009214384718457
  (1, 1396)	0.3375600812230469
  (1, 1200)	0.4904995132577479
  (2, 13771)	0.38505168207348944
  (2, 11754)	0.32106313955963073
  (2, 11142)	0.45862337972873657
  :	:
  (20797, 7744)	0.0868281349953222
  (20797, 7306)	0.18580559202052793
  (20797, 5979)	0.08188165916518761
  (20797, 5429)	0.31422059592151225
  (20797, 4372)	0.2940127910277199
  (20797, 3842)	

# splitting the data to training and testing data

In [36]:
x_train , x_test , y_train , y_test = train_test_split (x , y , test_size=0.2 , stratify= y , random_state=2)

# model training 

In [37]:
model = LogisticRegression()

In [38]:
model.fit(x_train , y_train)

LogisticRegression()

# evaluation

In [39]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction , y_train)

In [40]:
print(' accuracy score prediction ', training_data_accuracy)

 accuracy score prediction  0.9831730769230769


In [41]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction , y_test)

In [42]:
print(' accuracy score prediction ', test_data_accuracy)

 accuracy score prediction  0.9752403846153846


# making predictive system

In [44]:
x_new = x_test[1]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
    print('the news is real')
else:
    print('the news is fake ')

[0]
the news is real


In [45]:
print(y_test[1])

0
